### Schedule

In [6]:
for clock_idx in range(n_microbatches+n_partritions-1):
    from_partrition = max(clock_idx+1-n_microbatches, 0)
    to_partrition = min(clock_idx+1, n_partritions)
    
    print(f"from_partrition={from_partrition}") # ignore
    print(f"to_partrition={to_partrition}") # ignore

    results = []
    for partrition_idx in range(from_partrition, to_partrition):
        print(f"partrition_idx = {partrition_idx}") # ignore
        microbatch_idx = clock_idx-partrition_idx
        result = (microbatch_idx+1, partrition_idx+1)
        results.append(result)
    
    print(results)
    
    print("---------") # ignore

from_partrition=0
to_partrition=1
partrition_idx = 0
[(1, 1)]
---------
from_partrition=0
to_partrition=2
partrition_idx = 0
partrition_idx = 1
[(2, 1), (1, 2)]
---------
from_partrition=0
to_partrition=3
partrition_idx = 0
partrition_idx = 1
partrition_idx = 2
[(3, 1), (2, 2), (1, 3)]
---------
from_partrition=0
to_partrition=3
partrition_idx = 0
partrition_idx = 1
partrition_idx = 2
[(4, 1), (3, 2), (2, 3)]
---------
from_partrition=1
to_partrition=3
partrition_idx = 1
partrition_idx = 2
[(4, 2), (3, 3)]
---------
from_partrition=2
to_partrition=3
partrition_idx = 2
[(4, 3)]
---------


##### Example 1

In [3]:
n_microbatches = 4

In [4]:
n_partritions = 3

In [17]:
n_microbatches, n_partritions

(4, 3)

Reimplement the execution scheduler for pipeline parallelism in `TorchGPipe`.

In [18]:
for clock_idx in range(n_microbatches+n_partritions-1):
    from_partrition = max(clock_idx+1-n_microbatches, 0)
    to_partrition = min(clock_idx+1, n_partritions)

    tasks = []
    for partrition_idx in range(from_partrition, to_partrition):
        microbatch_idx = clock_idx-partrition_idx
        task = (microbatch_idx+1, partrition_idx+1)
        tasks.append(task)
    
    print(f"Clock cycle {clock_idx}: {tasks}") # ignore

Clock cycle 0: [(1, 1)]
Clock cycle 1: [(2, 1), (1, 2)]
Clock cycle 2: [(3, 1), (2, 2), (1, 3)]
Clock cycle 3: [(4, 1), (3, 2), (2, 3)]
Clock cycle 4: [(4, 2), (3, 3)]
Clock cycle 5: [(4, 3)]


### X

In [ ]:
import torch
import torch.nn as nn

# Define the subnetworks (partitions) of the neural network
f1 = nn.Linear(2, 4)
f2 = nn.ReLU()
f3 = nn.Linear(4, 1)

# Full network as a sequence of subnetworks
f = nn.Sequential(f1, f2, f3)

# Input data as a mini-batch
x = torch.tensor([[1, 1], [2, 2], [3, 3], [4, 4]], dtype=torch.float32)

In [ ]:
# Divide the mini-batch into micro-batches
m = 2
micro_batches = torch.chunk(x, m)

# Forward pass on micro-batches
outputs = []

In [ ]:
micro_batches

(tensor([[1., 1.],
         [2., 2.]]),
 tensor([[3., 3.],
         [4., 4.]]))

In [ ]:
for i in range(m):
    x_i = micro_batches[i]
    x_i_1 = f1(x_i)
    x_i_2 = f2(x_i_1)
    x_i_3 = f3(x_i_2)
    outputs.append(x_i_3)

# Aggregate the outputs
f_x = torch.cat(outputs, dim=0)
print("Forward pass result:", f_x)

Forward pass result: tensor([[0.1838],
        [0.2771],
        [0.3704],
        [0.4637]], grad_fn=<CatBackward0>)


In [ ]:
f_x

tensor([[0.1838],
        [0.2771],
        [0.3704],
        [0.4637]], grad_fn=<CatBackward0>)

In [ ]:
f(x)

tensor([[0.1838],
        [0.2771],
        [0.3704],
        [0.4637]], grad_fn=<AddmmBackward0>)

##### Example 1

In [ ]:
x = torch.tensor([[1, 1], [2, 2], [3, 3], [4, 4]], dtype=torch.float32)

In [ ]:
# Define the subnetworks (partitions) of the neural network
f1 = nn.Linear(2, 4)
f2 = nn.ReLU()
f3 = nn.Linear(4, 1)

# Full network as a sequence of subnetworks
model = nn.Sequential(f1, f2, f3)

In [ ]:
n_micros = 2

In [ ]:
import torch

In [ ]:
x

tensor([[1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.]])

In [ ]:
model

Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4, out_features=1, bias=True)
)

In [ ]:
output = model(x)

In [ ]:
n_micros

2

Implement a vanila pipeline parallelism with `n_micros` micro-batches. Explain

**Step 1**: First we split the batch into micro-batches

In [ ]:
micro_batches = torch.chunk(x, n_micros)

In [ ]:
outputs = []

In [ ]:
for i in range(n_micros):
    x_i = micro_batches[i]
    for subnetwork in model:
        x_i = subnetwork(x_i)
    
    outputs.append(x_i)

In [ ]:
parallel_output = torch.cat(outputs, dim=0)

In [ ]:
output == parallel_output

tensor([[True],
        [True],
        [True],
        [True]])

In [ ]:
x = torch.tensor([[-1, 2, 3], [4, 5, 6]])

In [ ]:
x[0]

tensor([-1,  2,  3])

In [ ]:
x[0]

tensor([-1,  2,  3])

In [ ]:
y = torch.tensor([-2, -1, 0, 1, 2])

In [ ]:
y.abs().sum()

tensor(6)

In [ ]:
y.pow(2).sum().sqrt()

tensor(3.1623)